## Categorization Manual

### Purpose
Purpose of notebook is to add more abstracted categories of emojis to the 'tweets_test_clean.json' file

In [1]:
## import all necessary packages
import json
import re
import pandas as pd

import nltk, string
from nltk.collocations import *

import numpy as np
import matplotlib.pyplot as plt

from pandas import *

from collections import defaultdict

import string
import happyfuntokenizing
import json

### Load Data

In [2]:
with open("./data/categories_manual.json") as json_file:
    category_dict = json.load(json_file)

In [3]:
with open('./data/emoji_webscraped_expanded.json','r') as f_emoji:
    emoji_df = DataFrame(json.load(f_emoji))

In [4]:
with open('./data/tweets_test_clean.json','r') as f:
    test_df = DataFrame(json.load(f))

In [5]:
with open('./data/tweets_training_clean.json','r') as f:
    training_df = DataFrame(json.load(f))

## Create array with faces used to create category column

In [6]:
def array_to_categorize(_dict):
    _unique = [face for item in _dict.values() for face in item]
    return _unique

## Add category column to dataframe

In [7]:
def cateogry_column(text, _unique):
    if len(text) == 0:
        return None
    
    boolean_face = [True if face in _unique else False for face in text]
    
    for item in boolean_face:
        if all(boolean_face) == True:
        
            cateogrize = []
            for category, faces in category_dict.items():
                for face in text:
                        if face in faces:
                            cateogrize.append(category)
            cateogrize = list(set(cateogrize))
            if len(cateogrize)==1:
                return cateogrize[0]
            else:
                return "Mix_P"
                
            return list(set(cateogrize))
        
        elif all(not element for element in boolean_face):
            return "Other"

        else:
            true_boolean = [index for index, item in enumerate(boolean_face) if item == True]
            keep_true = [text[i] for i in true_boolean]
            
            cateogrize = []
            for category, faces in category_dict.items():
                for face in keep_true:
                        if face in faces:
                            cateogrize.append(category)
            cateogrize = list(set(cateogrize))
            if len(cateogrize)==1:
                return cateogrize[0]
            else:
                return "Mix_P"

    return

## Make Final column numeric

In [8]:
def numeric_placeholder():
    _numeric = [(0, 'happy'), 
                (1, 'love'), 
                (2, 'despairing'), 
                (3, 'Other'), (4, 'Mix_P'), 
                (5, None), 
                (6, 'nonhuman')]
    return _numeric

In [9]:
def make_numeric(text, numeric_category):
    _num = [item[0] for item in numeric_category if text == item[1]]
    return _num[0]

In [10]:
def run_make_category(df, _dict, _numeric):
    unique_face = array_to_categorize(_dict)
    df["cateogry_prediction"] = df.only_emoji.apply(cateogry_column, _unique=unique_face)
    df['category_numeric'] = df.cateogry_prediction.apply(make_numeric, numeric_category=_numeric)
    return df

In [11]:
category_training_df = run_make_category(training_df, category_dict, numeric_placeholder())
category_training_df.to_json('./data/tweets_training_clean_preprocessing.json', force_ascii=False)

category_test_df = run_make_category(test_df, category_dict, numeric_placeholder())
category_test_df.to_json('./data/tweets_test_clean_preprocessing.json', force_ascii=False)